# **coding with google calab**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install tensorflow==2.3.0 coremltools==5.1.0 pillow==7.0.0 h5py==2.10.0

In [ ]:
!pip install keras==2.7.0

In [ ]:
import keras
import glob
import numpy as np
import keras
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.utils import np_utils #keras.utils.to_categoricalでエラーが出るので追加
from tensorflow.keras.optimizers import Adam # 「tensorflow.」を追加
import matplotlib.pyplot as plt
import time
import cv2
import random
import os
import coremltools
import coremltools as ct

In [ ]:
train_data_path = './drive/MyDrive/image_folder/total_triming_face_mask'

image_size = 25
color_setting = 3

In [ ]:
train_data_path = './drive/MyDrive/image_folder'

folder_name = os.listdir(train_data_path)
del folder_name[0]

folder_number = len(folder_name)

In [ ]:
x_images = []
y_labels = []

for index, name in enumerate(folder_name):
    read_data = train_data_path + '/' + name
    # print(read_data)
    if name == folder_name[0]:
        files = glob.glob(read_data + '/*.jpg')
    else :
        files = glob.glob(read_data + '/*.jpeg')
    # print(files)

    print(len(files))

    for i, file in enumerate(files): 
        #入力画像のカラーに合わせる

        img = load_img(file, color_mode = 'rgb' ,target_size=(image_size, image_size))  
        array = img_to_array(img)
        x_images.append(array)
        y_labels.append(index)

In [ ]:
x_images = np.array(x_images)
y_labels = np.array(y_labels)


In [ ]:
x_images = x_images.astype('float32') / 255

In [ ]:
index_count = len(y_labels)

count_array = []

for num in range(index_count):
    count_array.append(num)

random_count_array = random.sample(count_array, len(count_array))

x_images_shuffle = []
y_labels_shuffle = []

for i in range(index_count):
    y_labels_shuffle.append(y_labels[random_count_array[i]])
    x_images_shuffle.append(x_images[random_count_array[i]])

plt.imshow(x_images_shuffle[2])
plt.show()


In [ ]:
total_count = len(y_labels)
test_count = int(len(y_labels)/10) * 2
train_count = total_count - test_count

In [ ]:
x_train = []
y_train = []
x_test = []
y_test = []

x_train = x_images_shuffle[test_count + 1 : total_count - 1]
y_train = y_labels_shuffle[test_count + 1 : total_count - 1]
x_test = x_images_shuffle[0 : test_count]
y_test = y_labels_shuffle[0 : test_count]

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

y_train = np_utils.to_categorical(y_train, folder_number)
y_test = np_utils.to_categorical(y_test, folder_number)


In [ ]:
model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same',
          input_shape=(image_size, image_size, color_setting), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))               
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))                
model.add(Dropout(0.5))                                   
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))                                 
model.add(Dense(folder_number, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train, batch_size=50, epochs=20, verbose=1)

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.grid()
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.grid()
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Loss:', score[0], '（損失関数値 - 0に近いほど正解に近い）') 
print('Accuracy:', score[1] * 100, '%', '（精度 - 100% に近いほど正解に近い）') 

In [ ]:
recognise_image_path = './mask_image_sample.jpeg'

In [ ]:
img = cv2.imread(recognise_image_path, 1)  #ここを変更。モノクロ・グレースケールの場合は「0」。カラーの場合は「1」 。         
img = cv2.resize(img, (image_size, image_size))
plt.imshow(img)
plt.gray()  #ここを変更。カラーの場合は「plt.gray()」を消す。モノクロ・グレースケールの場合は「plt.gray()」が無いと変な色になります。
plt.show()

In [ ]:
img = img.reshape(image_size, image_size, color_setting).astype('float32')/255 

In [ ]:
prediction = model.predict(np.array([img]), verbose=1)
result = prediction[0]
result_index = np.argmax(result)
print(result)
print(result_index)
print(folder_name[result_index])

In [ ]:
new_dir_path = './model'
os.makedirs(new_dir_path, exist_ok=True)

In [ ]:
model.save('./model/mark_classification.h5')

In [ ]:
image_labels = [
    'mask_off', 
    'mask_on',
]

classifier_config = ct.ClassifierConfig(image_labels)
image_input = ct.ImageType(shape=(1, 25, 25, 3), scale=1/255)

mlmodel = ct.convert("./model/mark_classification.h5",  
                     inputs=[image_input],
                     classifier_config=classifier_config
                     )

mlmodel.save('./model/mask_model.mlmodel')

In [ ]:
coreml_model_path = './model/mask_model.mlmodel'
spec = coremltools.utils.load_spec(coreml_model_path)
builder = coremltools.models.neural_network.NeuralNetworkBuilder(spec=spec)
builder.inspect_input_features()